# Semantic baseline
Данный ноутбук реализует решение с исползованием семантических моделей ранжирования.

In [1]:
import tensorflow_hub as hub
import tensorflow_text
import numpy as np
from sklearn import metrics
import tensorflow as tf
import os
import pickle
from tqdm.notebook import tqdm
from collections import defaultdict, OrderedDict
import requests

Скачиваем universal sentence encoder

In [2]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3")

In [3]:
PATH = "/Users/michelle/data/nn_text-relevance-competition-ir-1-ts-fall-2020/"

In [4]:
listdir = os.listdir(os.path.join(PATH, "parsed_content"))

Читаем заголовки из файлов, которые были получены кодом nn_parser.py.

In [5]:
titles = {}
bad_files = []
for file in tqdm(listdir, total=len(listdir)):
    try:
        docid = file.split("_")[1].split(".")[0]
        with open(os.path.join(PATH, "parsed_content", file), "rb") as f:
            titles[docid] = pickle.load(f)[1]
    except:
        bad_files.append(file)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.




Traceback (most recent call last):
  File "/Users/michelle/Library/Python/3.7/lib/python/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-5-9da8ada4c0c7>", line 3, in <module>
    for file in tqdm(listdir, total=len(listdir)):
  File "/Users/michelle/Library/Python/3.7/lib/python/site-packages/tqdm/notebook.py", line 234, in __iter__
    for obj in super(tqdm_notebook, self).__iter__(*args, **kwargs):
  File "/Users/michelle/Library/Python/3.7/lib/python/site-packages/tqdm/std.py", line 1184, in __iter__
    self.refresh(lock_args=self.lock_args)
  File "/Users/michelle/Library/Python/3.7/lib/python/site-packages/tqdm/std.py", line 1380, in refresh
    self.display()
  File "/Users/michelle/Library/Python/3.7/lib/python/site-packages/tqdm/notebook.py", line 152, in display
    pbar.value = self.n
  File "/Users/michelle/Library/Python/3.7/lib/python/site-packages/traitlets/traitlets.py"

KeyboardInterrupt: 

In [ ]:
bad_files

In [ ]:
len(titles)

In [16]:
bad_parsed = {}
for file in tqdm(bad_files, total=len(bad_files)):
    docid = file.split("_")[1].split(".")[0]
    with open(os.path.join(PATH, "parsed_content", file), "rb") as f:
        bad_parsed[docid] = pickle.load(f)[1]

In [17]:
bad_parsed

{}

In [18]:
for key, value in bad_parsed.items():
    titles[key] = value

In [37]:
len(titles)

38114

Читаем запросы, sample и создаем словарь запрос-документы.

In [38]:
with open(PATH + "queries.numerate.txt", "r") as f:
    queries = f.read().split("\n")

In [6]:
with open(PATH + "sample.technosphere.ir1.textrelevance.submission.txt", "r") as f:
    sample = f.read().split("\n")

In [7]:
query_docid = defaultdict(list)
for item in sample[1:]:
    if item:
        qid = item.split(",")[0]
        docid = item.split(",")[1]
        query_docid[qid].append(docid)

In [41]:
queries_to_map = []

for query in queries[:-1]:
    queries_to_map.append(query.split("\t")[1])

Прогоняем запросы через спеллчекер, считаем для них эмбеддинги и сохраняем в файл.

In [42]:
def spellcheck(text):
    payload = {"text": text}
    response = requests.get("https://speller.yandex.net/services/spellservice.json/checkText",
                            params=payload).json()
    
    fixed_text = text
    for item in response:
        fixed_text = fixed_text.replace(item["word"], item["s"][0])
    return fixed_text

In [25]:
fix_queries_to_map = [spellcheck(query) for query in tqdm(queries_to_map, total=len(queries_to_map))]

In [32]:
queries_embeddings = embed(fix_queries_to_map)

In [33]:
import os

In [34]:
with open(os.path.join(PATH, "queries_embeddings"), "wb") as f:
    pickle.dump(queries_embeddings, f)

Создаем список все заголовков, затем итерируемся по запросам. Для каждого запроса по списку docid получаем эмбединги заголовков соответствующих документов и сохраняем в файл под номером запроса. Используем ограничение в 500 символов, так как встречаются аномально длинные заголовки, на которых модель падает.

In [49]:
titles_list = []
for i in range(1, len(titles) + 1):
    titles_list.append(titles[str(i)])

In [50]:
len(titles_list)

38114

In [15]:
!mkdir /Users/michelle/data/nn_text-relevance-competition-ir-1-ts-fall-2020/title_embeddings

In [51]:
with open(os.path.join(PATH, "titles.pkl"), "wb") as f:
    pickle.dump(titles, f)

In [8]:
with open(os.path.join(PATH, "titles.pkl"), "rb") as f:
    titles = pickle.load(f)

In [16]:
bad_queries = []
for query, docids in tqdm(query_docid.items(), total=len(query_docid)):
    try:
        titles_tmp = [titles[docid][:500] for docid in docids]
        titles_emb_tmp = embed(titles_tmp)
        with open(os.path.join(PATH, "title_embeddings", f"{query}.pkl"), "wb") as f:
            pickle.dump(titles_emb_tmp, f)
    except:
        print(query)
        bad_queries.append(query)

In [17]:
listdir = os.listdir(os.path.join(PATH, "title_embeddings"))

In [18]:
len(listdir)

399

In [20]:
with open(os.path.join(PATH, "queries_embeddings"), "rb") as f:
    queries_embeddings = pickle.load(f)

Снова итерируемся по запросам. Для каждого запроса читаем из файла эмбединги заголовков его документов. Затем считаем косинус между эмбедингом запроса и заголовка для каждого документа. По полученным значениям ранжируем список документов.

In [61]:
ranged_query_docid = {}
for query, docids in tqdm(query_docid.items(), total=len(query_docid)):
    with open(os.path.join(PATH, f"title_embeddings/{query}.pkl"), "rb") as f:
        data = pickle.load(f)
    ranged_query_docid[query] = np.array(docids)[np.argsort(metrics.pairwise.cosine_distances(tf.reshape(queries_embeddings[int(query) - 1], (1, -1)), data)[0])]

In [63]:
for query in ranged_query_docid:
    ranged_query_docid[query] = list(ranged_query_docid[query])

In [65]:
submit = ['QueryId,DocumentId']
for i in range(1, len(ranged_query_docid) + 1):
    docids = ranged_query_docid[str(i)]
    for docid in docids:
        submit.append(f"{i},{docid}")
        
with open(PATH + "submit_09_2.txt", "w") as f:
    f.write("\n".join(submit))